In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Préparation des fichiers résultats circonscriptions / Communes

In [3]:
# Charger les fichiers des résultats par circonscription et par commune:
data_circon = pd.read_csv('resultats-definitifs-par-circonscription.csv', sep=";")
data_commun = pd.read_csv('resultats-definitifs-par-commune.csv', sep=";", dtype={'Code commune': str, 'Code département': str})

# Renommer les colonnes pour correspondance au df 'gdf'
data_circon.rename(columns={'Code circonscription législative': 'codeCirconscription',
                            'Libellé circonscription législative':'nomCirconscription','Libellé département':'nomDepartement'}, inplace=True)
data_commun.rename(columns={'Libellé département':'nomDepartement'}, inplace=True)

data_circon.info(1)
data_commun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 322 columns):
 #    Column                      Dtype 
---   ------                      ----- 
 0    Code département            object
 1    nomDepartement              object
 2    codeCirconscription         object
 3    nomCirconscription          object
 4    Inscrits                    int64 
 5    Votants                     int64 
 6    % Votants                   object
 7    Abstentions                 int64 
 8    % Abstentions               object
 9    Exprimés                    int64 
 10   % Exprimés/inscrits         object
 11   % Exprimés/votants          object
 12   Blancs                      int64 
 13   % Blancs/inscrits           object
 14   % Blancs/votants            object
 15   Nuls                        int64 
 16   % Nuls/inscrits             object
 17   % Nuls/votants              object
 18   Numéro de panneau 1         int64 
 19   Nuance liste 1             

In [4]:
#--------------------------------------------- Mise en valeurs numériques des colonnes 'pourcentage' ---------------------------

# Colonnes récurrentes (par liste)
colonnes1 = [f'% Voix/exprimés {i}' for i in range(1, 39)]
colonnes2 = [f'% Voix/inscrits {i}' for i in range(1, 39)]
colonnes = colonnes1 + colonnes2
# Transformation de chaque colonne
for col in colonnes:
    data_circon[col] = data_circon[col].str.replace(',', '.').str.rstrip('%').astype('float')
    data_commun[col] = data_commun[col].str.replace(',', '.').str.rstrip('%').astype('float')

# Colonnes fixes
colonnes = [
    '% Votants', '% Abstentions', '% Exprimés/inscrits', '% Exprimés/votants', 
    '% Blancs/inscrits', '% Blancs/votants', '% Nuls/inscrits', '% Nuls/votants'
]
for col in colonnes:
    data_circon[col] = data_circon[col].str.replace(',', '.').str.rstrip('%').astype('float')
    data_commun[col] = data_commun[col].str.replace(',', '.').str.rstrip('%').astype('float')

data_commun.info()
data_circon.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35228 entries, 0 to 35227
Columns: 322 entries, Code département to Sièges 38
dtypes: float64(84), int64(120), object(118)
memory usage: 86.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 322 columns):
 #    Column                      Dtype  
---   ------                      -----  
 0    Code département            object 
 1    nomDepartement              object 
 2    codeCirconscription         object 
 3    nomCirconscription          object 
 4    Inscrits                    int64  
 5    Votants                     int64  
 6    % Votants                   float64
 7    Abstentions                 int64  
 8    % Abstentions               float64
 9    Exprimés                    int64  
 10   % Exprimés/inscrits         float64
 11   % Exprimés/votants          float64
 12   Blancs                      int64  
 13   % Blancs/inscrits           float64
 14   % Blancs/votants    

# Résultat National suivant les Circonscriptions et suivant les communes

In [6]:
# Sélection des colonnes utiles pour les listes de candidats et des voix
Selected_columns_nuance = [f'Libellé abrégé de liste {i}' for i in range(1, 39)]
Selected_columns_voix = [f'Voix {i}' for i in range(1, 39)]
Selected_columns_candidats = [f'Libellé de liste {i}' for i in range(1, 39)]
Selected_columns_vote = [f'% Voix/exprimés {i}' for i in range(1, 39)]
Selected_columns_vote_inscrit = [f'% Voix/inscrits {i}' for i in range(1, 39)]

# Extraction des données
# Circoncription
List_Candidats = data_circon[Selected_columns_candidats].iloc[0].tolist()
Voix_Candidats = data_circon[Selected_columns_voix].sum().tolist()
nuance_Candidats = data_circon[Selected_columns_nuance].iloc[0].tolist()
Vote_Candidats = data_circon[Selected_columns_vote].mean().tolist()
Vote_inscrits_Candidats = data_circon[Selected_columns_vote_inscrit].mean().tolist()
# Communes
List_Candidats1 = data_commun[Selected_columns_candidats].iloc[0].tolist()
Voix_Candidats1 = data_commun[Selected_columns_voix].sum().tolist()
nuance_Candidats1 = data_commun[Selected_columns_nuance].iloc[0].tolist()
Vote_Candidats1 = data_commun[Selected_columns_vote].mean().tolist()
Vote_inscrits_Candidats1 = data_commun[Selected_columns_vote_inscrit].mean().tolist()

# Création des DataFrames finaux
# Circoncription
Df = pd.DataFrame({
    'Candidats': List_Candidats,
    'Nuance' : nuance_Candidats,
    'Nb de Voix': Voix_Candidats,
    '% voix_exprimées':Vote_Candidats,
    '% voix_inscrits':Vote_inscrits_Candidats
}, index=range(1, len(List_Candidats) + 1))
# Communes
Df1 = pd.DataFrame({
    'Candidats': List_Candidats1,
    'Nuance' : nuance_Candidats1,
    'Nb de Voix': Voix_Candidats1,
    '% voix_exprimées':Vote_Candidats1,
    '% voix_inscrits':Vote_inscrits_Candidats1
}, index=range(1, len(List_Candidats1) + 1))

# Classement par nombre de voix
# Circoncription
Df = Df.sort_values(by='Nb de Voix', ascending=False).round(2)
# Communes
Df1 = Df1.sort_values(by='Nb de Voix', ascending=False).round(2)

# Affichage des DataFrames
# Circoncription
Resultat_Circon = Df.reset_index()
Resultat_Circon.rename(columns={'index': 'No Liste'},inplace=True)
display(Resultat_Circon)
# Communes
Resultat_Commune= Df1.reset_index()
Resultat_Commune.rename(columns={'index': 'No Liste'},inplace=True)
display(Resultat_Commune)

,No Liste,Candidats,Nuance,Nb de Voix,% voix_exprimées,% voix_inscrits
0,5,LA FRANCE REVIENT ! AVEC JORDAN BARDELLA ET MA...,La FRANCE REVIENT,7765936,31.03,15.60
1,11,BESOIN D'EUROPE,BESOIN D'EUROPE,3614646,14.54,7.25
2,27,RÉVEILLER L'EUROPE,REVEIL EUR,3424216,13.52,6.90
3,4,LA FRANCE INSOUMISE - UNION POPULAIRE,LFI - UP,2448703,10.76,5.16
4,18,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,1794171,7.12,3.63
5,6,EUROPE ÉCOLOGIE,EUROPE ÉCOLOGIE,1361883,5.45,2.74
6,3,"LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET ...","LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET ...",1353127,5.48,2.72
7,33,GAUCHE UNIE POUR LE MONDE DU TRAVAIL SOUTENUE ...,GAUCHE UNIE,584067,2.34,1.18
8,29,ALLIANCE RURALE,AR,582901,2.26,1.17
9,8,"PARTI ANIMALISTE - LES ANIMAUX COMPTENT, VOTRE...",PARTI ANIMALISTE,495936,1.95,1.00


,No Liste,Candidats,Nuance,Nb de Voix,% voix_exprimées,% voix_inscrits
0,5,LA FRANCE REVIENT ! AVEC JORDAN BARDELLA ET MA...,La FRANCE REVIENT,7765936,38.28,21.47
1,11,BESOIN D'EUROPE,BESOIN D'EUROPE,3614646,13.12,7.35
2,27,RÉVEILLER L'EUROPE,REVEIL EUR,3424216,11.34,6.43
3,4,LA FRANCE INSOUMISE - UNION POPULAIRE,LFI - UP,2448703,5.40,2.96
4,18,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,1794171,7.36,4.18
5,6,EUROPE ÉCOLOGIE,EUROPE ÉCOLOGIE,1361883,4.24,2.40
6,3,"LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET ...","LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET ...",1353127,5.40,3.03
7,33,GAUCHE UNIE POUR LE MONDE DU TRAVAIL SOUTENUE ...,GAUCHE UNIE,584067,2.44,1.38
8,29,ALLIANCE RURALE,AR,582901,4.91,2.83
9,8,"PARTI ANIMALISTE - LES ANIMAUX COMPTENT, VOTRE...",PARTI ANIMALISTE,495936,2.13,1.20


# Les 4 listes arrivées en-tête par Circonscriptions et par Communes

In [8]:
Nb_Lists = 5
def top_4_listes(data):
    # Création des listes des colonnes de voix et des listes correspondantes
    voix_columns = [col for col in data.columns if col.startswith("% Voix/exprimés")]
    Listes_columns = ['Libellé abrégé de liste ' + col.split()[2] for col in voix_columns]

    # Vérification que les colonnes existent
    assert all(col in data.columns for col in voix_columns), "Certaines colonnes de voix sont manquantes dans le DataFrame."
    assert all(col in data.columns for col in Listes_columns), "Certaines colonnes de listes sont manquantes dans le DataFrame."

    # Fonction pour obtenir les 4 listes ayant obtenues le plus de voix
    def Top_Listes(row):
        # Combiner les listes et les voix en paires
        votes_result = [(row[Listes], row[Voix]) for Listes, Voix in zip(Listes_columns, voix_columns)]
        # Trier les %votes par ordre décroissant et prendre les 4 premiers
        top_4 = sorted(votes_result, key=lambda x: x[1], reverse=True)[:Nb_Lists]
        # Réorganiser pour l'alternance Liste, %Voix
        alternated = []
        for Listes, Voix in top_4:
            alternated.extend([Listes, Voix])
        return pd.Series(alternated)

    # Appliquer la fonction sur le DataFrame
    top_4_results = data.apply(Top_Listes, axis=1)

    # Générer les noms des colonnes alternées Parti, Voix
    column_names = []
    for i in range(len(top_4_results.columns) // 2):
        column_names.append(f'Liste{i+1}')
        column_names.append(f'% Voix Liste{i+1}')
    top_4_results.columns = column_names

    # Conserver les 18 premières colonnes du DataFrame original
    columns_conserve = data.iloc[:, :18]

    # Concaténer les résultats
    Resultat = pd.concat([columns_conserve, top_4_results], axis=1)

    return Resultat

# Appliquer la fonction aux deux DataFrames
Resultat_Comm = top_4_listes(data_commun)
Resultat_Circon = top_4_listes(data_circon)


# Sélection des colonnes numériques utilisant une expression régulière
colonnes_numeriques = [col for col in Resultat_Circon.columns if col.startswith('% Voix Liste')]
# Création de la colonne 'somme' en additionnant les valeurs des colonnes numériques
Resultat_Circon['somme'] = Resultat_Circon[colonnes_numeriques].sum(axis=1)
Resultat_Circon = Resultat_Circon.sort_values(by="somme", ascending=True)


# Afficher les premières lignes des résultats
display(Resultat_Comm.head(3))
Resultat_Circon.head(3)


,Code département,nomDepartement,Code commune,Libellé commune,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Liste1,% Voix Liste1,Liste2,% Voix Liste2,Liste3,% Voix Liste3,Liste4,% Voix Liste4,Liste5,% Voix Liste5
0,01,Ain,01001,L'Abergement-Clémenciat,662,373,56.34,289,43.66,369,...,La FRANCE REVIENT,42.82,BESOIN D'EUROPE,14.36,"LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET ...",8.13,REVEIL EUR,7.32,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,6.50
1,01,Ain,01002,L'Abergement-de-Varey,228,156,68.42,72,31.58,150,...,La FRANCE REVIENT,20.00,BESOIN D'EUROPE,14.67,LFI - UP,13.33,EUROPE ÉCOLOGIE,13.33,REVEIL EUR,12.00
2,01,Ain,01004,Ambérieu-en-Bugey,8708,4627,53.14,4081,46.86,4531,...,La FRANCE REVIENT,34.54,LFI - UP,13.62,REVEIL EUR,12.43,BESOIN D'EUROPE,10.84,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,6.49


,Code département,nomDepartement,codeCirconscription,nomCirconscription,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,% Voix Liste1,Liste2,% Voix Liste2,Liste3,% Voix Liste3,Liste4,% Voix Liste4,Liste5,% Voix Liste5,somme
576,ZZ,Français établis hors de France,ZZ11,11ème circonscription,104696,22800,21.78,81896,78.22,22555,...,23.64,REVEIL EUR,16.48,EUROPE ÉCOLOGIE,10.76,"LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET ...",10.36,La FRANCE REVIENT,9.48,70.72
326,64,Pyrénées-Atlantiques,6405,5ème circonscription,101389,55259,54.50,46130,45.50,53848,...,23.97,REVEIL EUR,18.23,BESOIN D'EUROPE,16.24,EUROPE ÉCOLOGIE,6.92,LFI - UP,6.49,71.85
327,64,Pyrénées-Atlantiques,6406,6ème circonscription,108254,57188,52.83,51066,47.17,55531,...,22.08,BESOIN D'EUROPE,17.78,REVEIL EUR,16.93,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,8.21,EUROPE ÉCOLOGIE,6.86,71.86


In [9]:
Resultat_Circon.to_excel("Résultats_Circon_10ere_Listes.xlsx", index=True)

In [10]:
Resultat_Circon.info(1)

<class 'pandas.core.frame.DataFrame'>
Index: 577 entries, 576 to 49
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Code département     577 non-null    object 
 1   nomDepartement       577 non-null    object 
 2   codeCirconscription  577 non-null    object 
 3   nomCirconscription   577 non-null    object 
 4   Inscrits             577 non-null    int64  
 5   Votants              577 non-null    int64  
 6   % Votants            577 non-null    float64
 7   Abstentions          577 non-null    int64  
 8   % Abstentions        577 non-null    float64
 9   Exprimés             577 non-null    int64  
 10  % Exprimés/inscrits  577 non-null    float64
 11  % Exprimés/votants   577 non-null    float64
 12  Blancs               577 non-null    int64  
 13  % Blancs/inscrits    577 non-null    float64
 14  % Blancs/votants     577 non-null    float64
 15  Nuls                 577 non-null    int64  

In [11]:
Resultat_Circon[Resultat_Circon['codeCirconscription'] == "0702"]

,Code département,nomDepartement,codeCirconscription,nomCirconscription,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,% Voix Liste1,Liste2,% Voix Liste2,Liste3,% Voix Liste3,Liste4,% Voix Liste4,Liste5,% Voix Liste5,somme
27,07,Ardèche,0702,2ème circonscription,97365,56648,58.18,40717,41.82,54811,...,35.49,BESOIN D'EUROPE,13.97,REVEIL EUR,12.99,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,7.75,LFI - UP,6.52,76.72


# LEGISLATIVES 2022 - Les 3 listes arrivées en-tête par Circonscriptions

In [13]:
Data2022 = pd.read_excel("Legis2022-Circons.xlsx")

In [14]:
Data2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   COD_CIRCONS     550 non-null    object 
 1   Inscrits        550 non-null    int64  
 2   Abstentions     550 non-null    int64  
 3    % Abs/Ins      550 non-null    float64
 4   Votants         550 non-null    int64  
 5   % Vot/Ins       550 non-null    float64
 6   Blancs          550 non-null    int64  
 7   Blancs/Ins      550 non-null    float64
 8   % Blancs/Vot    550 non-null    float64
 9   Nuls            550 non-null    int64  
 10  % Nuls/Ins      550 non-null    float64
 11  % Nuls/Vot      550 non-null    float64
 12  Exprimés        550 non-null    int64  
 13  % Exp/Ins       550 non-null    float64
 14  % Exp/Vot       550 non-null    float64
 15  DIV Voix        550 non-null    int64  
 16  DIV % Voix/Exp  550 non-null    float64
 17  DSV Voix        550 non-null    int

Nb_Lists = 3
def top_4_listes(data):
    # Création des listes des colonnes de voix et des listes correspondantes
    voix_columns = [col for col in data.columns if col.endswith("% Voix/Exp")]

    # Vérification que les colonnes existent
    assert all(col in data.columns for col in voix_columns), "Certaines colonnes de voix sont manquantes dans le DataFrame."

    # Fonction pour obtenir les 4 listes ayant obtenues le plus de voix
    def Top_Listes(row):
        # Combiner les listes et les voix en paires
        votes_result = [row[Voix] for Voix in zip(voix_columns)]
        # Trier les %votes par ordre décroissant et prendre les 4 premiers
        top_4 = sorted(votes_result, key=lambda x: x[1], reverse=True)[:Nb_Lists]
       
    # Appliquer la fonction sur le DataFrame
    top_4_results = data.apply(Top_Listes, axis=1)

    # Conserver les 18 premières colonnes du DataFrame original
    columns_conserve = data.iloc[:, :15]

    # Concaténer les résultats
    Resultat = pd.concat([columns_conserve, top_4_results], axis=1)

    return Resultat

# Appliquer la fonction aux deux DataFrames
Resultat_2022 = top_4_listes(Data2022)

In [16]:
Nb_Lists = 3

def top_3_listes(data):
    # Création des listes des colonnes de voix et des listes correspondantes
    voix_columns = [col for col in data.columns if col.endswith("% Voix/Exp")]

    # Fonction pour obtenir les 3 listes ayant obtenues le plus de voix
    def Top_Listes(row):
        # Créer une liste de tuples (nom de colonne, valeur)
        votes_result = [(col, row[col]) for col in voix_columns]
        # Trier les tuples par valeur décroissante et prendre les 3 premiers
        top_3 = sorted(votes_result, key=lambda x: x[1], reverse=True)[:Nb_Lists]
        # Extraire uniquement les noms de colonnes des 3 meilleurs
        top_3_colnames = [col for col, val in top_3]
        # Extraire uniquement les valeurs des 3 meilleurs
        top_3_values = [val for col, val in top_3]
        # Combiner les noms de colonnes et les valeurs dans un dictionnaire
        result = {}
        for i in range(Nb_Lists):
            result[f"Top_{i+1}_List"] = top_3_colnames[i]
            result[f"Top_{i+1}_%"] = top_3_values[i]
        return pd.Series(result)
       
    # Appliquer la fonction sur le DataFrame
    top_3_results = data.apply(Top_Listes, axis=1)

    # Conserver les 15 premières colonnes du DataFrame original
    columns_conserve = data.iloc[:, :15]

    # Concaténer les résultats
    Resultat = pd.concat([columns_conserve, top_3_results], axis=1)

    return Resultat

Resultat_2022 = top_3_listes(Data2022)

# Afficher les résultats
Resultat_2022


,COD_CIRCONS,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,Blancs/Ins,% Blancs/Vot,Nuls,...,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%
0,0101,86187,43652,49.306930,42535,50.693070,490,0.586842,1.165789,234,...,0.534737,41811,49.836140,98.299649,LR % Voix/Exp,25.43,NUP % Voix/Exp,23.01,RN % Voix/Exp,22.94
1,0102,98818,49731,49.657596,49087,50.342500,619,0.650673,1.292692,166,...,0.342596,48302,49.520000,98.364615,ENS % Voix/Exp,25.77,NUP % Voix/Exp,24.87,RN % Voix/Exp,23.75
2,0103,82204,45946,52.908636,36258,47.091364,385,0.647636,1.332182,144,...,0.555273,35729,46.149273,98.112455,ENS % Voix/Exp,28.19,NUP % Voix/Exp,23.07,LR % Voix/Exp,17.92
3,0104,94897,49258,50.662672,45639,49.337414,640,0.736810,1.499914,207,...,0.443190,44792,48.383793,98.056724,RN % Voix/Exp,25.52,LR % Voix/Exp,16.89,NUP % Voix/Exp,16.89
4,0105,77456,40382,47.157338,37074,52.842662,820,1.375468,2.502662,206,...,0.609209,36048,51.133453,96.888273,NUP % Voix/Exp,24.32,RN % Voix/Exp,19.64,DVD % Voix/Exp,18.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,9707,238760,168793,71.126593,69967,28.873462,1359,0.900824,3.273187,1307,...,3.653681,67301,27.001593,93.073132,NUP % Voix/Exp,24.94,DVC % Voix/Exp,12.13,ENS % Voix/Exp,10.72
546,9708,131219,115176,83.964167,16043,16.035833,180,0.245000,1.565833,16,...,0.122500,15847,15.773333,98.313333,ENS % Voix/Exp,27.31,UDI % Voix/Exp,23.15,NUP % Voix/Exp,22.74
547,9709,120556,102826,84.742778,17730,15.257222,392,0.357222,2.686111,70,...,0.566111,17268,14.843889,96.750000,NUP % Voix/Exp,40.20,ENS % Voix/Exp,25.91,DVD % Voix/Exp,6.38
548,9710,104303,83943,78.094894,20360,21.905106,290,0.407021,1.868511,45,...,0.175319,20025,21.451064,97.956170,NUP % Voix/Exp,33.08,ENS % Voix/Exp,28.05,LR % Voix/Exp,9.35


In [17]:
Resultat_2022.to_excel("3eresListes2022.xlsx", index=True)

# Prévision Législatives 2024

In [19]:
Prev2024=pd.read_excel("PrevisionsLegis_2024.xlsx")

In [20]:
Prev2024=Prev2024.iloc[0:539]
Prev2024

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Union de la gauche,Nb VOIX UG,%VOIX UG,Union de l'extrême droite,Nb VOIX Union EXTD,%VOIX EXTD,Union des Démocrates et Indépendants,Nb VOIX UDI,%VOIX UDI,TOTAL %VOIX/exp
0,01,Ain,0101,1ère circonscription,86824,48408,0.5575,38416,0.4425,46887,...,1,10269.6,0.21908,0,0.0,0.00000,0,0.0,0.00000,0.90465
1,01,Ain,0102,2ème circonscription,102088,58219,0.5703,43869,0.4297,56962,...,1,12685.6,0.22278,0,0.0,0.00000,0,0.0,0.00000,0.91254
2,01,Ain,0103,3ème circonscription,85884,42940,0.5000,42944,0.5000,41880,...,1,11276.4,0.26926,0,0.0,0.00000,0,0.0,0.00000,0.89808
3,01,Ain,0104,4ème circonscription,96186,53221,0.5533,42965,0.4467,51624,...,1,9664.6,0.18724,0,0.0,0.00000,0,0.0,0.00000,0.90664
4,01,Ain,0105,5ème circonscription,78235,41472,0.5301,36763,0.4699,40170,...,1,7141.2,0.17772,1,17440.6,0.43423,0,0.0,0.00000,0.92438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,95,Val-d'Oise,9506,6ème circonscription,76344,38641,0.5061,37703,0.4939,37942,...,1,14517.6,0.38268,0,0.0,0.00000,0,0.0,0.00000,0.95270
535,95,Val-d'Oise,9507,7ème circonscription,68907,34055,0.4942,34852,0.5058,33430,...,1,11097.6,0.33202,0,0.0,0.00000,0,0.0,0.00000,0.95600
536,95,Val-d'Oise,9508,8ème circonscription,56146,21369,0.3806,34777,0.6194,20784,...,1,10943.2,0.52672,0,0.0,0.00000,1,2197.6,0.10574,0.92988
537,95,Val-d'Oise,9509,9ème circonscription,72109,30759,0.4266,41350,0.5734,29992,...,1,11438.4,0.38150,0,0.0,0.00000,0,0.0,0.00000,0.95190


In [21]:
Nb_Lists = 5

def top_3_listes(data):
    # Création des listes des colonnes de voix et des listes correspondantes
    voix_columns = [col for col in data.columns if col.startswith("%VOIX")]

    # Fonction pour obtenir les 3 listes ayant obtenues le plus de voix
    def Top_Listes(row):
        # Créer une liste de tuples (nom de colonne, valeur)
        votes_result = [(col, row[col]) for col in voix_columns]
        # Trier les tuples par valeur décroissante et prendre les 3 premiers
        top_3 = sorted(votes_result, key=lambda x: x[1], reverse=True)[:Nb_Lists]
        # Extraire uniquement les noms de colonnes des 3 meilleurs
        top_3_colnames = [col for col, val in top_3]
        # Extraire uniquement les valeurs des 3 meilleurs
        top_3_values = [val for col, val in top_3]
        # Combiner les noms de colonnes et les valeurs dans un dictionnaire
        result = {}
        for i in range(Nb_Lists):
            result[f"Top_{i+1}_List"] = top_3_colnames[i]
            result[f"Top_{i+1}_%"] = top_3_values[i]
        return pd.Series(result)
       
    # Appliquer la fonction sur le DataFrame
    top_3_results = data.apply(Top_Listes, axis=1)

    # Conserver les 15 premières colonnes du DataFrame original
    columns_conserve = data.iloc[:, :18]

    # Concaténer les résultats
    Resultat = pd.concat([columns_conserve, top_3_results], axis=1)

    return Resultat

Resultat_2024 = top_3_listes(Prev2024)

# Afficher les résultats
Resultat_2024

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
0,01,Ain,0101,1ère circonscription,86824,48408,0.5575,38416,0.4425,46887,...,%VOIX RN,0.43223,%VOIX UG,0.21908,%VOIX Ensemble,0.21706,%VOIX Les Républicains,0.03088,%VOIX Ext Gauche,0.00540
1,01,Ain,0102,2ème circonscription,102088,58219,0.5703,43869,0.4297,56962,...,%VOIX RN,0.42522,%VOIX DIVC,0.22806,%VOIX UG,0.22278,%VOIX Les Républicains,0.03168,%VOIX Ext Gauche,0.00480
2,01,Ain,0103,3ème circonscription,85884,42940,0.5000,42944,0.5000,41880,...,%VOIX RN,0.33492,%VOIX UG,0.26926,%VOIX Ensemble,0.26006,%VOIX Les Républicains,0.02884,%VOIX Ext Gauche,0.00500
3,01,Ain,0104,4ème circonscription,96186,53221,0.5533,42965,0.4467,51624,...,%VOIX RN,0.46598,%VOIX Ensemble,0.21658,%VOIX UG,0.18724,%VOIX Les Républicains,0.03184,%VOIX Ext Gauche,0.00500
4,01,Ain,0105,5ème circonscription,78235,41472,0.5301,36763,0.4699,40170,...,%VOIX EXTD,0.43423,%VOIX Ensemble,0.18670,%VOIX UG,0.17772,%VOIX Ecologie,0.05070,%VOIX Reconquête,0.04179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,95,Val-d'Oise,9506,6ème circonscription,76344,38641,0.5061,37703,0.4939,37942,...,%VOIX UG,0.38268,%VOIX RN,0.26631,%VOIX Ensemble,0.22360,%VOIX Reconquête,0.04165,%VOIX Les Républicains,0.03316
535,95,Val-d'Oise,9507,7ème circonscription,68907,34055,0.4942,34852,0.5058,33430,...,%VOIX UG,0.33202,%VOIX RN,0.32626,%VOIX Ensemble,0.19970,%VOIX Reconquête,0.06188,%VOIX Les Républicains,0.03044
536,95,Val-d'Oise,9508,8ème circonscription,56146,21369,0.3806,34777,0.6194,20784,...,%VOIX UG,0.52672,%VOIX RN,0.25305,%VOIX UDI,0.10574,%VOIX Reconquête,0.03507,%VOIX Ext Gauche,0.00930
537,95,Val-d'Oise,9509,9ème circonscription,72109,30759,0.4266,41350,0.5734,29992,...,%VOIX UG,0.38150,%VOIX RN,0.35885,%VOIX Ensemble,0.14880,%VOIX Reconquête,0.03605,%VOIX Les Républicains,0.01960


In [22]:
Resultat_2024.loc[(Resultat_2024["Top_1_%"] >= 0.5) & (Resultat_2024["Top_1_List"] == "%VOIX RN")]

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
5,02,Aisne,0201,1ère circonscription,72186,37844,0.5243,34342,0.4757,36630,...,%VOIX RN,0.54820,%VOIX Ensemble,0.17034,%VOIX UG,0.15566,%VOIX Reconquête,0.03220,%VOIX Ext Gauche,0.00900
6,02,Aisne,0202,2ème circonscription,73361,36721,0.5006,36640,0.4994,35625,...,%VOIX RN,0.54313,%VOIX DIVC,0.17096,%VOIX UG,0.15618,%VOIX Reconquête,0.03745,%VOIX Les Républicains,0.02988
8,02,Aisne,0204,4ème circonscription,79145,38493,0.4864,40652,0.5136,37440,...,%VOIX RN,0.55457,%VOIX Ensemble,0.16524,%VOIX UG,0.16256,%VOIX Ext Gauche,0.00830,%VOIX DIVC,0.00000
9,02,Aisne,0205,5ème circonscription,82853,42808,0.5167,40045,0.4833,41447,...,%VOIX RN,0.53223,%VOIX Ensemble,0.17028,%VOIX UG,0.16910,%VOIX Les Républicains,0.02536,%VOIX Ext Gauche,0.00830
20,06,Alpes-Maritimes,0604,4ème circonscription,87017,45894,0.5274,41123,0.4726,44568,...,%VOIX RN,0.51804,%VOIX Ensemble,0.16972,%VOIX UG,0.12682,%VOIX Ecologie,0.03370,%VOIX Ext Gauche,0.00330
29,08,Ardennes,0801,1ère circonscription,70836,37220,0.5254,33616,0.4746,35955,...,%VOIX RN,0.50621,%VOIX Ensemble,0.19088,%VOIX UG,0.12956,%VOIX Ecologie,0.03520,%VOIX Les Républicains,0.02988
30,08,Ardennes,0802,2ème circonscription,61088,28567,0.4676,32521,0.5324,27672,...,%VOIX RN,0.51514,%VOIX UG,0.19000,%VOIX Ensemble,0.16410,%VOIX Ext Gauche,0.00880,%VOIX DIVC,0.00000
31,08,Ardennes,0803,3ème circonscription,54945,28663,0.5217,26282,0.4783,27481,...,%VOIX RN,0.52094,%VOIX DIVC,0.18022,%VOIX UG,0.14450,%VOIX Ext Gauche,0.00890,%VOIX Divers Gauche,0.00000
34,10,Aube,1001,1ère circonscription,64182,35223,0.5488,28959,0.4512,34097,...,%VOIX RN,0.54582,%VOIX Ensemble,0.18642,%VOIX UG,0.13250,%VOIX Les Républicains,0.03504,%VOIX Ext Gauche,0.00500
54,13,Bouches-du-Rhône,1312,12ème circonscription,95395,48996,0.5136,46399,0.4864,47992,...,%VOIX RN,0.53471,%VOIX UG,0.20894,%VOIX Ensemble,0.14474,%VOIX Reconquête,0.04725,%VOIX Les Républicains,0.01776


In [23]:
Resultat_2024.loc[(Resultat_2024["Top_1_%"] >= 0.5) & (Resultat_2024["Top_1_List"] == "%VOIX UG")]

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
46,13,Bouches-du-Rhône,1304,4ème circonscription,67418,30120,0.4468,37298,0.5532,29756,...,%VOIX UG,0.52572,%VOIX RN,0.15952,%VOIX Ensemble,0.13578,%VOIX Ecologie,0.10430,%VOIX Reconquête,0.02198
49,13,Bouches-du-Rhône,1307,7ème circonscription,69020,25900,0.3753,43120,0.6247,25460,...,%VOIX UG,0.56694,%VOIX RN,0.29348,%VOIX Ensemble,0.07212,%VOIX Reconquête,0.02394,%VOIX Ext Gauche,0.00500
156,34,Hérault,3402,2ème circonscription,67835,33345,0.4916,34490,0.5084,32879,...,%VOIX UG,0.52972,%VOIX Ensemble,0.20756,%VOIX RN,0.17073,%VOIX Les Républicains,0.01856,%VOIX Ext Gauche,0.00630
274,59,Nord,5901,1ère circonscription,61501,32115,0.5222,29386,0.4778,31652,...,%VOIX UG,0.51374,%VOIX Ensemble,0.20380,%VOIX RN,0.19554,%VOIX Ext Gauche,0.00770,%VOIX DIVC,0.00000
399,75,Paris,7515,15ème circonscription,80317,45686,0.5688,34631,0.4312,45176,...,%VOIX UG,0.57980,%VOIX Ensemble,0.22102,%VOIX RN,0.11534,%VOIX Reconquête,0.02450,%VOIX Les Républicains,0.01904
402,75,Paris,7518,18ème circonscription,71087,42825,0.6024,28262,0.3976,42450,...,%VOIX UG,0.52508,%VOIX Ensemble,0.28660,%VOIX RN,0.10491,%VOIX Les Républicains,0.02396,%VOIX Reconquête,0.02345
493,92,Hauts-de-Seine,9201,1ère circonscription,68219,29712,0.4355,38507,0.5645,29050,...,%VOIX UG,0.58790,%VOIX RN,0.17307,%VOIX Ensemble,0.14242,%VOIX Reconquête,0.02177,%VOIX Les Républicains,0.01584
506,93,Seine-Saint-Denis,9301,1ère circonscription,67959,29318,0.4314,38641,0.5686,28759,...,%VOIX UG,0.52924,%VOIX Ensemble,0.15748,%VOIX RN,0.13825,%VOIX Ecologie,0.09010,%VOIX Les Républicains,0.01364
507,93,Seine-Saint-Denis,9302,2ème circonscription,59952,21991,0.3668,37961,0.6332,21463,...,%VOIX UG,0.62528,%VOIX RN,0.14484,%VOIX Ensemble,0.10632,%VOIX Ecologie,0.05710,%VOIX Ext Gauche,0.01170
509,93,Seine-Saint-Denis,9304,4ème circonscription,68036,24223,0.3560,43813,0.6440,23488,...,%VOIX UG,0.60554,%VOIX RN,0.21290,%VOIX Ensemble,0.09482,%VOIX Les Républicains,0.01092,%VOIX Ext Gauche,0.00920


In [24]:
Resultat_2024.loc[(Resultat_2024["Top_1_%"] >= 0.5) & (Resultat_2024["Top_1_List"] == "%VOIX EXT Droite")]

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
7,02,Aisne,0203,3ème circonscription,66183,33884,0.512,32299,0.488,32547,...,%VOIX EXT Droite,0.61051,%VOIX DIVC,0.14578,%VOIX Divers Gauche,0.11454,%VOIX Reconquête,0.03409,%VOIX Ext Gauche,0.0094


In [25]:
Resultat_2024.loc[(Resultat_2024["Top_1_%"] >= 0.5) & (Resultat_2024["Top_1_List"] == "%VOIX EXTD")] 

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
112,28,Eure-et-Loir,2804,4ème circonscription,67564,35435,0.5245,32129,0.4755,33925,...,%VOIX EXTD,0.51433,%VOIX Ensemble,0.19506,%VOIX UG,0.14814,%VOIX Reconquête,0.04277,%VOIX Ext Gauche,0.0074


# Choix % votants supplémentaires

In [27]:
Prev2024.iloc[:,17:,].head(5)

,% Nuls/votants,Divers centre,Nb VOIX DIVC,%VOIX DIVC,Divers gauche,Nb VOIX DIVG,%VOIX Divers Gauche,Ecologistes,Nb VOIX Ecologiques,%VOIX Ecologie,...,Union de la gauche,Nb VOIX UG,%VOIX UG,Union de l'extrême droite,Nb VOIX Union EXTD,%VOIX EXTD,Union des Démocrates et Indépendants,Nb VOIX UDI,%VOIX UDI,TOTAL %VOIX/exp
0,0.0163,0.0,0.0,0.00000,0,0.0,0.0,0,0,0.0000,...,1,10269.6,0.21908,0,0.0,0.00000,0,0.0,0.0,0.90465
1,0.0092,1.0,12989.4,0.22806,0,0.0,0.0,0,0,0.0000,...,1,12685.6,0.22278,0,0.0,0.00000,0,0.0,0.0,0.91254
2,0.0138,0.0,0.0,0.00000,0,0.0,0.0,0,0,0.0000,...,1,11276.4,0.26926,0,0.0,0.00000,0,0.0,0.0,0.89808
3,0.0154,0.0,0.0,0.00000,0,0.0,0.0,0,0,0.0000,...,1,9664.6,0.18724,0,0.0,0.00000,0,0.0,0.0,0.90664
4,0.0156,0.0,0.0,0.00000,0,0.0,0.0,1,2034,0.0507,...,1,7141.2,0.17772,1,17440.6,0.43423,0,0.0,0.0,0.92438


In [29]:
TauxVote = 0.2
Votants = Prev2024.iloc[:, :6]
Votants=Votants.rename(columns={"Votants": "Votants_EE"})
Votants["Votants"] = round(Votants["Votants_EE"] * TauxVote + Votants["Votants_EE"],0).astype(int)
Votants["% Votants"] = round((Votants["Votants"] * 100) / Votants["Inscrits"],2)
Votants["Abstentions"] = Votants["Inscrits"] - Votants["Votants"]
Votants["% Abstentions"] = round((Votants["Abstentions"] * 100) / Votants["Inscrits"],2)
Votants["% nuls/blancs"] = round((Prev2024["% Blancs/votants"] + Prev2024["% Nuls/votants"]) * 100,2)
Votants["nuls/blancs"] = round((Votants["% nuls/blancs"] * Votants["Votants"])/100,0).astype(int)
Votants.head(1)

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants_EE,Votants,% Votants,Abstentions,% Abstentions,% nuls/blancs,nuls/blancs
0,01,Ain,0101,1ère circonscription,86824,48408,58090,66.91,28734,33.09,3.14,1824


In [58]:
Resultat_2024.loc[(Resultat_2024["Top_1_%"] >= 0.5) & (Resultat_2024["Libellé circonscription législative"] == "19ème circonscription")]

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
292,59,Nord,5919,19ème circonscription,81105,39321,0.4848,41784,0.5152,38419,...,%VOIX RN,0.5636,%VOIX UG,0.22282,%VOIX Ensemble,0.1146,%VOIX Ext Gauche,0.0093,%VOIX DIVC,0.0


# Recherche par commune

In [56]:
Communes = pd.read_excel("Communes_Circons.xlsx", dtype={'COD_COM': str, 'CODE CANTON': str})
Communes.head()

,COD_DEP,NOM DPT,COD_COM,COD_Circons,NOM COMMUNE,CODE CANTON,NOM CANTON
0,01,Ain,016,0101,Arbigny,17,Replonges
1,01,Ain,024,0101,Attignat,2,Attignat
2,01,Ain,029,0101,Beaupont,18,Saint-Etienne-du-Bois
3,01,Ain,038,0101,Bény,18,Saint-Etienne-du-Bois
4,01,Ain,040,0101,Béréziat,2,Attignat


In [45]:
def Com_Ciro(ville):
    # Filtrer la commune dans le DataFrame Communes
    ligne = Communes[Communes["NOM COMMUNE"] == ville]

    if not ligne.empty:
        # Extraire le COD_CIRCO
        cod_circo = ligne["COD_Circons"].values[0]

        # Filtrer par le COD_CIRCO
        lignes_correspondantes = Resultat_2024[Resultat_2024["Code circonscription législative"] == cod_circo]

        # Lignes correspondantes
        return lignes_correspondantes
    else:
        return "Commune non trouvée"

In [47]:
Com_Ciro("Draveil")

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
491,91,Essonne,9109,9ème circonscription,80506,39904,0.4957,40602,0.5043,39151,...,%VOIX UG,0.34044,%VOIX RN,0.32367,%VOIX Ensemble,0.21798,%VOIX Ext Gauche,0.0053,%VOIX DIVC,0.0


In [49]:
Com_Ciro("Montargis")

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
219,45,Loiret,4504,4ème circonscription,74443,38443,0.5164,36000,0.4836,37310,...,%VOIX RN,0.50594,%VOIX UG,0.18346,%VOIX Ensemble,0.1811,%VOIX Reconquête,0.03598,%VOIX Les Républicains,0.02552


In [51]:
Com_Ciro("Hénin-Beaumont")

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
315,62,Pas-de-Calais,6211,11ème circonscription,94265,45632,0.4841,48633,0.5159,44465,...,%VOIX RN,0.57393,%VOIX UG,0.22084,%VOIX Ensemble,0.1127,%VOIX Reconquête,0.02275,%VOIX Les Républicains,0.01088


In [53]:
Com_Ciro("Bastia")

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
123,2B,Haute-Corse,2B01,1ère circonscription,63005,27051,0.4293,35954,0.5707,26435,...,%VOIX RN,0.46883,%VOIX REG,0.2077,%VOIX DIVC,0.17924,%VOIX Divers Gauche,0.17248,%VOIX Reconquête,0.05691
